# Data Processing

In [2]:
!git clone https://github.com/zygmuntz/goodbooks-10k

Cloning into 'goodbooks-10k'...
remote: Enumerating objects: 66, done.
remote: Total 66 (delta 0), reused 0 (delta 0), pack-reused 66
Unpacking objects: 100% (66/66), done.


In [3]:
%cd goodbooks-10k/

/kaggle/working/goodbooks-10k


# EDA

In [4]:
import random
random.seed(2001)

import pandas as pd
pd.set_option('display.max_columns', 20)
ratings = pd.read_csv("./ratings.csv")
books = pd.read_csv("./books.csv")
print("List column ratings: ", ratings.columns, "\nList column books", books.columns)
print(ratings.head())
print(books.head())
print(books.describe())

List column ratings:  Index(['user_id', 'book_id', 'rating'], dtype='object') 
List column books Index(['book_id', 'goodreads_book_id', 'best_book_id', 'work_id',
       'books_count', 'isbn', 'isbn13', 'authors', 'original_publication_year',
       'original_title', 'title', 'language_code', 'average_rating',
       'ratings_count', 'work_ratings_count', 'work_text_reviews_count',
       'ratings_1', 'ratings_2', 'ratings_3', 'ratings_4', 'ratings_5',
       'image_url', 'small_image_url'],
      dtype='object')
   user_id  book_id  rating
0        1      258       5
1        2     4081       4
2        2      260       5
3        2     9296       5
4        2     2318       3
   book_id  goodreads_book_id  best_book_id  work_id  books_count       isbn  \
0        1            2767052       2767052  2792775          272  439023483   
1        2                  3             3  4640799          491  439554934   
2        3              41865         41865  3212258          226  316015

In [5]:
print(ratings.shape)
ratings.iloc[:25]

(5976479, 3)


,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3
5,2,26,4
6,2,315,3
7,2,33,4
8,2,301,5
9,2,2686,5


In [6]:
print(ratings.shape)
print(ratings.describe())

(5976479, 3)
            user_id       book_id        rating
count  5.976479e+06  5.976479e+06  5.976479e+06
mean   2.622446e+04  2.006477e+03  3.919866e+00
std    1.541323e+04  2.468499e+03  9.910868e-01
min    1.000000e+00  1.000000e+00  1.000000e+00
25%    1.281300e+04  1.980000e+02  3.000000e+00
50%    2.593800e+04  8.850000e+02  4.000000e+00
75%    3.950900e+04  2.973000e+03  5.000000e+00
max    5.342400e+04  1.000000e+04  5.000000e+00


In [7]:
!pip install scikit-surprise

In [8]:
from surprise.dataset import Dataset
from surprise.reader import Reader
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(ratings, test_size=0.20, random_state=2001)
print('Training data shape: ', train_data.shape)
print('Testing data shape: ', test_data.shape)
test_data.head()

reader = Reader(rating_scale=(1,5))
train_dataset = Dataset.load_from_df(train_data, reader)
test_dataset = Dataset.load_from_df(test_data, reader)
dataset = Dataset.load_from_df(ratings, reader)

Training data shape:  (4781183, 3)
Testing data shape:  (1195296, 3)


# Training

In [9]:
from surprise.prediction_algorithms.random_pred import NormalPredictor
from surprise import accuracy
from surprise.model_selection import train_test_split
temp_train_dataset, temp_test_dataset = train_test_split(dataset, test_size=0.20, random_state=2001)

native = NormalPredictor()
native.fit(temp_train_dataset)

predictions = native.test(temp_test_dataset)
rmse = accuracy.rmse(predictions, verbose=True)
print('Root mean square error on Random: ', rmse)

RMSE: 1.3245
Root mean square error on Random:  1.3245017259736365


In [11]:
# trying with the training data after preprocessing 
from surprise import KNNWithMeans
from surprise.model_selection import GridSearchCV

sim_options = {
    "name": ["msd", "cosine"],
    "min_support": [3, 4],
    "user_based": [False],
}


param_grid = {"sim_options": sim_options}
grid_Search = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse", "mae"])
grid_Search.fit(train_dataset)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...


In [13]:
print(grid_Search.best_score["rmse"])
print(grid_Search.best_params["rmse"])
results_df = pd.DataFrame.from_dict(grid_Search.cv_results)
results_df.iloc[:,:]

0.8442119834779206
{'sim_options': {'name': 'msd', 'min_support': 4, 'user_based': False}}


,split0_test_rmse,split1_test_rmse,split2_test_rmse,split3_test_rmse,split4_test_rmse,mean_test_rmse,std_test_rmse,rank_test_rmse,split0_test_mae,split1_test_mae,...,split4_test_mae,mean_test_mae,std_test_mae,rank_test_mae,mean_fit_time,std_fit_time,mean_test_time,std_test_time,params,param_sim_options
0,0.845846,0.844707,0.845149,0.844007,0.844422,0.844826,0.000631,2,0.655748,0.654797,...,0.654667,0.654882,0.000580,2,26.352206,0.231211,214.641344,1.289403,"{'sim_options': {'name': 'msd', 'min_support':...","{'name': 'msd', 'min_support': 3, 'user_based'..."
1,0.845276,0.844020,0.844534,0.843451,0.843778,0.844212,0.000639,1,0.655026,0.653990,...,0.653854,0.654119,0.000574,1,26.263649,0.271755,212.393958,2.270011,"{'sim_options': {'name': 'msd', 'min_support':...","{'name': 'msd', 'min_support': 4, 'user_based'..."
2,0.849551,0.848342,0.848834,0.847633,0.848178,0.848507,0.000647,4,0.659345,0.658389,...,0.658419,0.658529,0.000539,4,50.085102,0.351394,213.348034,1.776055,"{'sim_options': {'name': 'cosine', 'min_suppor...","{'name': 'cosine', 'min_support': 3, 'user_bas..."
3,0.849425,0.848172,0.848666,0.847455,0.847977,0.848339,0.000668,3,0.659044,0.658070,...,0.658005,0.658185,0.000547,3,47.126178,0.363515,213.577038,1.862594,"{'sim_options': {'name': 'cosine', 'min_suppor...","{'name': 'cosine', 'min_support': 4, 'user_bas..."


In [14]:
from surprise import SVD

param_grid = {'n_epochs': [5, 10], 
              'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6],
              'random_state': [2001]}
grid_Search = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
grid_Search.fit(train_dataset)

In [16]:
print(grid_Search.best_score["rmse"])
print(grid_Search.best_params["rmse"])
results_df = pd.DataFrame.from_dict(grid_Search.cv_results)
results_df.iloc[:,:]

0.8680871630024529
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4, 'random_state': 2001}


,split0_test_rmse,split1_test_rmse,split2_test_rmse,mean_test_rmse,std_test_rmse,rank_test_rmse,split0_test_mae,split1_test_mae,split2_test_mae,mean_test_mae,...,rank_test_mae,mean_fit_time,std_fit_time,mean_test_time,std_test_time,params,param_n_epochs,param_lr_all,param_reg_all,param_random_state
0,0.894609,0.895291,0.895322,0.895074,0.000329,7,0.710932,0.711563,0.711756,0.711417,...,7,82.404203,0.166864,24.931643,1.487700,"{'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0....",5,0.002,0.4,2001
1,0.898966,0.899583,0.899654,0.899401,0.000309,8,0.713442,0.714000,0.714259,0.713900,...,8,80.839956,0.166694,24.235114,1.572642,"{'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0....",5,0.002,0.6,2001
2,0.873259,0.874024,0.873962,0.873748,0.000347,2,0.693165,0.693654,0.693739,0.693519,...,2,81.193676,0.183052,25.567029,1.760313,"{'n_epochs': 5, 'lr_all': 0.005, 'reg_all': 0....",5,0.005,0.4,2001
3,0.880035,0.880668,0.880731,0.880478,0.000314,5,0.699005,0.699403,0.699625,0.699344,...,5,80.637189,0.215388,23.471301,0.277102,"{'n_epochs': 5, 'lr_all': 0.005, 'reg_all': 0....",5,0.005,0.6,2001
4,0.876471,0.877345,0.877160,0.876992,0.000376,4,0.696282,0.696933,0.696914,0.696710,...,4,161.086934,0.250682,25.307461,1.519445,"{'n_epochs': 10, 'lr_all': 0.002, 'reg_all': 0...",10,0.002,0.4,2001
5,0.882768,0.883542,0.883428,0.883246,0.000341,6,0.701353,0.701912,0.701994,0.701753,...,6,161.054471,0.138867,25.406386,1.863227,"{'n_epochs': 10, 'lr_all': 0.002, 'reg_all': 0...",10,0.002,0.6,2001
6,0.867580,0.868435,0.868247,0.868087,0.000367,1,0.687413,0.687883,0.687866,0.687721,...,1,161.294449,0.469418,24.189611,1.550944,"{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0...",10,0.005,0.4,2001
7,0.875363,0.876065,0.876028,0.875819,0.000323,3,0.694942,0.695325,0.695471,0.695246,...,3,161.207256,0.198366,25.440711,1.562448,"{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0...",10,0.005,0.6,2001


In [20]:
sim_options = {
    "name": "msd",
    "min_support": 4,
    "user_based": False,
}

KNN = KNNWithMeans(sim_options=sim_options)
KNN.fit(temp_train_dataset)
predictions = KNN.test(temp_test_dataset)
accuracy.rmse(predictions, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8400


0.8399887009180161

In [22]:
# saving trained model
from surprise import dump
import os
model_filename = "./model.pickle"
# Dump algorithm and reload it.
file_name = os.path.expanduser(model_filename)
dump.dump(file_name, algo=KNN)
print(model_filename)

./model.pickle


In [24]:
## load saved model
def load_model(model_filename):
    file_name = os.path.expanduser(model_filename)
    _, loaded_model = dump.load(file_name)
    return loaded_model

In [30]:
ratings.head()

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3


In [36]:
print(temp_test_dataset[:10])

[(26750, 661, 5.0), (30107, 1056, 1.0), (2901, 99, 3.0), (46774, 2109, 5.0), (45759, 57, 5.0), (22114, 729, 5.0), (7253, 763, 3.0), (38375, 28, 3.0), (2459, 29, 3.0), (53034, 8718, 5.0)]


In [43]:
# predicitng
from pprint import pprint as pp
model_filename = "./model.pickle"
KNN_load_model = load_model(model_filename)
def itemRating(user, item, r_ui=None):
    prediction = KNN_load_model.predict(user, item, r_ui=r_ui, verbose=True)

In [44]:
itemRating(user=46774, item=57, r_ui=5)

user: 46774      item: 57         r_ui = 5.00   est = 4.55   {'actual_k': 40, 'was_impossible': False}


In [51]:
list_item = pd.unique(ratings['book_id']).tolist()

In [90]:
import numpy as np
# recommender
def recommed(user):
    rated = ratings[ratings['user_id'] == user]
    rated = rated['book_id']
    
    predict = lambda i: KNN_load_model.predict(user, i)
    prediction = np.array([predict(item) for item in list_item if item not in rated])
    prediction = prediction[prediction[:, 3].argsort()[::-1]]
    return prediction[:10]

In [93]:
recommed(3)

array([[3, 9950, None, 4.340909090909091,
        {'actual_k': 0, 'was_impossible': False}],
       [3, 7835, None, 4.274725274725275,
        {'actual_k': 0, 'was_impossible': False}],
       [3, 8967, None, 4.243589743589744,
        {'actual_k': 0, 'was_impossible': False}],
       [3, 8145, None, 4.230769230769231,
        {'actual_k': 0, 'was_impossible': False}],
       [3, 9580, None, 4.229508196721311,
        {'actual_k': 0, 'was_impossible': False}],
       [3, 9836, None, 4.21875, {'actual_k': 0, 'was_impossible': False}],
       [3, 9913, None, 4.205882352941177,
        {'actual_k': 0, 'was_impossible': False}],
       [3, 8892, None, 4.202702702702703,
        {'actual_k': 0, 'was_impossible': False}],
       [3, 8344, None, 4.190476190476191,
        {'actual_k': 0, 'was_impossible': False}],
       [3, 9548, None, 4.148936170212766,
        {'actual_k': 0, 'was_impossible': False}]], dtype=object)

In [95]:
SVD_Modal = SVD(
    n_epochs=30, 
    lr_all=0.005,
    reg_all=0.4,
    random_state=2001,
    verbose=True)
SVD_Modal.fit(temp_train_dataset)
predictions = SVD_Modal.test(temp_test_dataset)
accuracy.rmse(predictions, verbose=True)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29
RMSE: 0.8662


0.8661968814947723